<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Выводы:" data-toc-modified-id="Выводы:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Выводы:</a></span></li></ul></li></ul></div>

#  Разработка алгоритма защиты персональных данных


***Цель исследования:***
* *Защита данных клиентов страховой компании.*

***Задачи исследования***
* *Разработка метода преобразования данных, для сложного восстановления персональной информации;*
* *Преобразование данных без ухудшения качества моделей машинного обучения.*


## Загрузка данных

In [1]:
# импортируем необходимые для работы бибилиотеки
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression


In [2]:
# прочитаем и сохраним данные
df = pd.read_csv('/datasets/insurance.csv')


In [3]:
# выведем информацию о располагаемых данных
print(f'\nОбъектов: {df.shape[0]}')
print(f'Признаков: {df.shape[1]}')
print(f'Пропусков: {df.isna().sum().sum()}')
print(f'Дубликатов: {df.duplicated().sum()}')
print('\n\nПервые объекты и признаки:')
display(df.head())



Объектов: 5000
Признаков: 5
Пропусков: 0
Дубликатов: 153


Первые объекты и признаки:


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# выведем на экран количество уникальных значений признаков
pd.DataFrame(df.nunique(), columns=['Количество уникальных значений'])


,Количество уникальных значений
Пол,2
Возраст,46
Зарплата,524
Члены семьи,7
Страховые выплаты,6


In [5]:
# выведем на экран таблицу корреляции признаков
df.corr()


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


In [6]:
# проверим взаимосвязь возраста и количества страховых выплат
payments = (
    df.groupby(['Страховые выплаты', 'Возраст'])
    .agg({'Возраст': 'count'})
    .rename(columns={'Возраст': 'Количество застрахованных'}))

payments

Количество застрахованных
Страховые выплаты Возраст                           
0                 18.0                           117
                  19.0                           223
                  20.0                           195
                  21.0                           200
                  22.0                           209
                  23.0                           202
                  24.0                           182
                  25.0                           214
                  26.0                           211
                  27.0                           209
                  28.0                           204
                  29.0                           203
                  30.0                           202
                  31.0                           212
                  32.0                           206
                  33.0                           191
                  34.0                           177
                  35.0                           179
                  36.0                           193
                  37.0                           147
                  38.0                           139
                  39.0                           141
                  40.0                           114
                  41.0                           129
                  42.0                            37
1                 42.0                            56
                  43.0                            77
                  44.0                            74
                  45.0                            73
                  46.0                            60
                  47.0                            47
                  48.0                            26
                  49.0                            10
2                 49.0                            27
                  50.0                            27
                  51.0                            21
                  52.0                            22
                  53.0                            11
                  54.0                             7
3                 55.0                             9
                  56.0                             5
                  57.0                             2
                  58.0                             2
4                 59.0                             3
                  60.0                             2
                  61.0                             1
                  62.0                             1
5                 65.0                             1

### Выводы
* Данные загружены и изучены, необходимости в предобработке - не выявлено (пропуски отсутствуют, мультиколлинеарности между признаками нет);
* Датасет содержит 153 полных дубликата, которые не будут удалены ввиду маленького количества признаков и их уникальных значений (вероятно, дубликаты - совпадения признаков различных объектов);
* Выявлена средняя корреляция (0,65) между возрастом и количеством страховых выплат за последние 5 лет, корреляция между остальными признаками - отсутствует


## Умножение матриц


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

***Вопрос:***

*Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?*

***Ответ:***

*Качество линейной регрессии - не изменится*


***Обоснование:***

Исходные предсказания:  
$$
a = Xw
$$

Предсказания после умножения признаков на обратимую матрицу:  
$$
a_P = XPw_P
$$

Подставим формулу обучения вместо $w_P$:  
$$
a_P = XP ((XP)^T XP)^{-1} (XP)^T y
$$

Раскроем скобки, воспользовавшись свойствами матриц:
* свойства операции транспонирования матриц: $(AB)^T = B^T A^T$
* свойства обратной матрицы: $(AB)^{-1} = B^{-1}A^{-1}$
* свойства операции умножения матриц: $A(BC) = (AB)C$
* свойство умножения матрицы на единичную: $AE = EA = A$
* свойство умножения матрицы на обратную: $AA^{-1} = A^{-1}A = E$

$$
a_P = XP (P^T X^T XP)^{-1} P^T X^T y
$$
$$
a_P = XP P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
a_P = X (X^T X)^{-1} X^T y
$$

Подставим формулу обучения:  
$$
a_P = Xw
$$

Что и требовалось доказать:
$$
a_P = a
$$


## Алгоритм преобразования

***Алгоритм преобразования данных:***
* Умножение матрицы признаков на случайную обратимую матрицу

***Обоснование:***
* Данный способ позволит выполнить задачу исследования - преобразовать исходные данные пользователей без ухудшения качества модели машинного обучения
* Доказательство приведено в п.2 настоящего исследования


## Проверка алгоритма

In [7]:
# выделим признаки и ключевой признак
features = df.drop('Страховые выплаты', axis='columns')
target = df['Страховые выплаты']

display(features.head())
target.head()


,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


0    0
1    1
2    0
3    0
4    0
Name: Страховые выплаты, dtype: int64

In [8]:
# обучим модель и выведем значение метрики R2
model_initial = LinearRegression()
model_initial.fit(features, target)
r2_initial = model_initial.score(features, target)

print(f'Значение метрики R2: {r2_initial}')


Значение метрики R2: 0.4249455028666801


In [9]:
# создадим случайную квадратную обратимую матрицу
# т.е. определитель которой != 0:
det = 0
while det == 0:
    key = np.random.randn(features.shape[1], features.shape[1])
    det = np.linalg.det(key)
key

print(key)


[[ 3.92682975e-01 -8.45662847e-01 -1.26967480e+00 -1.13138490e+00]
 [ 1.66954959e+00 -1.46918204e-03 -5.05906348e-01 -5.78454100e-01]
 [-4.65550507e-01 -1.30435924e-01 -2.47722067e-01 -1.63898395e+00]
 [-1.56769815e+00 -1.13109317e+00  2.25829932e-01 -7.89264277e-01]]


In [10]:
# преобразуем признаки и посчитаем значения метрики R2
features_new = features @ key
model_new = LinearRegression()
model_new.fit(features_new, target)
r2_new = model_new.score(features_new, target)


In [11]:
# выведем на экран сравнение метрик R2 до и после преобразования признаков
index = ['Модель с исходными признаками', 'Модель с преобразованными признаками']

display(pd.DataFrame({'R2': [r2_initial, r2_new]}, index=index))


,R2
Модель с исходными признаками,0.424946
Модель с преобразованными признаками,0.424946


In [13]:
# для дешифровки признаков необходимо зашифрованные данные умножить на матрицу,
# обратную той что использовалась для шифрования
import math
b = features_new @ np.linalg.inv(key)
for column in b.columns:
    b[column] = b[column].apply(lambda x: int(round(x, 0)))

b.columns=['Пол', 'Возраст', 'Зарплата', 'Члены семьи']

print('Исходные признаки:')
display(features.head(10))

print('Признаки после дешифровки:')
display(b.head(10))


Исходные признаки:


,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
5,1,43.0,41000.0,2
6,1,39.0,39700.0,2
7,1,25.0,38600.0,4
8,1,36.0,49700.0,1
9,1,32.0,51700.0,1


Признаки после дешифровки:


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0
5,1,43,41000,2
6,1,39,39700,2
7,1,25,38600,4
8,1,36,49700,1
9,1,32,51700,1


### Выводы:
Задачи проекта выполнены:
* метод домножения матрицы признаков на случайную обратимую матрицу - позволил обеспечить защиту данных пользователей;
* данные преобразованы без ухудшения качества модели машинного обучения.